### Importing packages and data_utils.py

In [1]:
import sys
import os
sys.path.append('../')

In [27]:
os.path.join(os.getcwd()  , '..')

'/Users/samart/Desktop/LEAP/Climsim/ClimSim_Playground/preprocessing/..'

In [6]:
sys.path

['/Users/samart/Desktop/LEAP/Climsim/ClimSim_Playground/preprocessing',
 '/Users/samart/anaconda3/envs/leap/lib/python311.zip',
 '/Users/samart/anaconda3/envs/leap/lib/python3.11',
 '/Users/samart/anaconda3/envs/leap/lib/python3.11/lib-dynload',
 '',
 '/Users/samart/anaconda3/envs/leap/lib/python3.11/site-packages',
 '/Users/samart/anaconda3/envs/leap/lib/python3.11/site-packages/PyNIO']

In [1]:
from climsim_utils.data_utils import *

2023-10-18 01:08:01.087221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Instantiating class

In [2]:
grid_info

NameError: name 'grid_info' is not defined

In [6]:
grid_path = '../grid_info/ClimSim_low-res_grid-info.nc'
norm_path = 'normalizations/'

grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + 'inputs/input_mean.nc')
input_max = xr.open_dataset(norm_path + 'inputs/input_max.nc')
input_min = xr.open_dataset(norm_path + 'inputs/input_min.nc')
output_scale = xr.open_dataset(norm_path + 'outputs/output_scale.nc')

data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale)

### Create training data

In [7]:
# set data path
data.data_path = 'Data/'

# set inputs and outputs to V1 subset
data.set_to_v1_vars()

# v1 inputs (name :: description :: dimension :: units): 

# 'state_t' :: air temperature :: 60 :: K 
# 'state_q0001' :: specific humidity :: 60 :: kg/kg
# 'state_ps' :: surface pressure :: 1 :: Pa
# 'pbuf_SOLIN' :: solar insolation :: 1 :: W/m^2
# 'pbuf_LHFLX' :: surface latent heat flux :: 1 :: W/m^2
# 'pbuf_SHFLX' :: surface sensible heat flux :: 1 :: W/m^2

# v1 outputs (name :: description :: dimension :: units): 

# 'ptend_t' :: heating tendency :: 60 :: K/s 
# 'ptend_q0001' :: moistening tendency :: 60 :: kg/kg/s
# 'cam_out_NETSW' :: net shortwave flux at surface :: 1 :: W/m^2
# 'cam_out_FLWDS' :: downward longwave flux at surface :: 1 :: W/m^2 
# 'cam_out_PRECSC' :: snow rate (liquid water equivalent) :: 1 :: m/s 
# 'cam_out_PRECC' :: rain rate :: 1 :: m/s
# 'cam_out_SOLS' :: downward visible direct solar flux to surface :: 1 :: W/m^2
# 'cam_out_SOLL' :: downward near-infrared direct solar flux to surface :: 1 :: W/m^2
# 'cam_out_SOLSD' :: downward diffuse solar flux to surface :: 1 :: W/m^2
# 'cam_out_SOLLD' :: downward diffuse near-infrared solar flux to surface :: 1 :: W/m^2

# set regular expressions for selecting training data
data.set_regexps(data_split = 'train', 
                 regexps = ['E3SM-MMF.mli.000[1234567]-*-*-*.nc', # years 1 through 7
                            'E3SM-MMF.mli.0008-01-*-*.nc']) # first month of year 8
# set temporal subsampling
data.set_stride_sample(data_split = 'train', stride_sample = 7)
# create list of files to extract data from
data.set_filelist(data_split = 'train')
# save numpy files of training data
data.save_as_npy(data_split = 'train', save_path = '')

2023-10-18 01:09:26.392976: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-18 01:09:26.393605: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-10-18 01:09:26.540750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


ValueError: need at least one array to concatenate

### Create validation data

In [8]:
# set regular expressions for selecting validation data
data.set_regexps(data_split = 'val',
                 regexps = ['E3SM-MMF.mli.0008-0[23456789]-*-*.nc', # months 2 through 9 of year 8
                            'E3SM-MMF.mli.0008-1[012]-*-*.nc', # months 10 through 12 of year 8
                            'E3SM-MMF.mli.0009-01-*-*.nc']) # first month of year 9
# set temporal subsampling
data.set_stride_sample(data_split = 'val', stride_sample = 7)
# create list of files to extract data from
data.set_filelist(data_split = 'val')
# save numpy files of validation data
data.save_as_npy(data_split = 'val', save_path = '')

2023-10-18 01:09:50.640960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


ValueError: need at least one array to concatenate

### Create scoring data

In [9]:
# set regular expressions for selecting scoring data (stride of 6 is needed for daily averaging)
data.set_regexps(data_split = 'scoring',
                 regexps = ['E3SM-MMF.mli.0008-0[23456789]-*-*.nc', # months 2 through 9 of year 8
                            'E3SM-MMF.mli.0008-1[012]-*-*.nc', # months 10 through 12 of year 8
                            'E3SM-MMF.mli.0009-01-*-*.nc']) # first month of year 9
# set temporal subsampling
data.set_stride_sample(data_split = 'scoring', stride_sample = 6)
# create list of files to extract data from
data.set_filelist(data_split = 'scoring')
# save numpy files of scoring data
data.save_as_npy(data_split = 'scoring', save_path = '')

2023-10-18 01:09:54.512936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


ValueError: need at least one array to concatenate

In [10]:
print('finished')

finished
